# Library Import


In [9]:
import wave
import numpy as np
import matplotlib.pyplot as plt
from Crypto.Hash import SHA3_256
from Crypto.Cipher import ChaCha20_Poly1305
from Crypto.Random import get_random_bytes

# SHA-3 Digest


In [10]:
aliceKey = get_random_bytes(500)
print("Alice's Key: ", aliceKey)
hashObject = SHA3_256.new()
hashObject.update(aliceKey)
key = hashObject.digest()
print("Hash of Alice's key: ", key)

Alice's Key:  b'l\x04v\xea\x81y\xdb\x86F\xe0\x04\xba\xe9\xd2\x97\xbdDS\xb6q\xf5\\iY\xe0\x1f\xa4\x84\x08\x90}Pu\x88\x19V\x90W\xffIC\x8e\xe2\x002{\xd3\n\x05N\x96\\\xc9y)\xd4u\xbc\xb4\xe8(f\x00\xa9\xa5$^\x07\xee\xab\x9a[\x02D6\x9eZ"q\xef\x1f\xc1\x10\x81"\xe1\x7f\x03\x89e\x17\xef\xbdU\x8d\x92\xffs\xb8n\\\t\xa4\xa2\x934-\xcfd\x05\xe1n\xaeP.z\x138\x89\xa3+\xc1\xfas\xc7\x15O\x8d\xe1\xc1\xb7\x99\xa6\x96\xa0w]\xff\xefxwU\xc3\xb7^\xbb\xa1\xf0\xa4\xe7n\xc9\t.\xbe\x05\xd4\xfc\xe0\xcb\xde\x1dG\x92f\x13\x03r\x04\x93\xac\xb4di\x9e5\x89i\x12F\xde\xfa7dd\xa0\xf8\xf807bd\x15\xc2*D\xd2W\x83X\xecP\x8f\xda\xc3\xc4}l\xe5\x95\r\xaap,ic\n\xee\x00\x9d6\xa3\x02\xd1\xe2\xc7\x00\xe7\x0f\x01Z\x90\x97X\xc8\xbf\xec\xce6\xd3k0\xe8\xe1\xf7\xce\xed\xcc\x9d`$\x06\x9b\xf1\x95\xa4\x14\x89\x1e\xaa\x14#Ms\x1c\xe7\xb2L\xd0\xd6\x17O\xc9\xf3W\xa8\x0b\xcb\xb2\x8cJ\xc5\xfe^\xfd\x03\xed\xa1\x85vJ\xf1,\x0c\xf0\xfcFJ5\xb3\x97\xb6\xc8JK\xcb\xc2)F\xe9\x1cz\xadp\x88\xb8\xe8,8\x99\xfb-\xf5\xc5m\x87*\xc4\x9d\x05D\x9a\xd1\x81M"\xfd(s~\x8

# Steganography


## Function Definition


### Embedding


#### Audio Clipping Function


In [11]:
def clip_audio(audio_data, min_value=-32768, max_value=32767):
    return np.clip(audio_data, min_value, max_value)

#### Audio Embedding Function


In [12]:
def embed_audio(carrier_path, secret_path, output_path, num_bits=8):
    # Open the carrier and secret audio files
    carrier = wave.open(carrier_path, 'rb')
    secret = wave.open(secret_path, 'rb')
    # Ensure compatibility between carrier and secret audio
    if carrier.getnchannels() != secret.getnchannels() or carrier.getsampwidth() != secret.getsampwidth():
        raise ValueError(
            "Carrier and secret audio must have the same number of channels and sample width.")
    # Read audio data
    carrier_frames = np.frombuffer(carrier.readframes(
        carrier.getnframes()), dtype=np.int16)
    secret_frames = np.frombuffer(secret.readframes(
        secret.getnframes()), dtype=np.int16)
    # Ensure carrier has enough capacity to store secret
    max_secret_length = len(carrier_frames) // (16 // num_bits)
    if len(secret_frames) > max_secret_length:
        raise ValueError(f"Carrier audio cannot hold the secret audio. Max capacity: {
                         max_secret_length}, got: {len(secret_frames)}.")
    # Resize the secret frames to match embedding capacity
    padded_secret_frames = np.zeros(max_secret_length, dtype=np.int16)
    padded_secret_frames[:len(secret_frames)] = secret_frames
    # Embed the secret audio into the carrier
    embedded_frames = np.copy(carrier_frames)
    mask = (1 << num_bits) - 1  # Mask to isolate the LSBs
    embedded_frames &= ~mask  # Clear the least significant bits
    embedded_frames[:len(padded_secret_frames)] += (padded_secret_frames >>
                                                    # Embed secret bits
                                                    (16 - num_bits)) & mask
    # Clip audio values to valid range
    embedded_frames = clip_audio(embedded_frames)
    # Write the embedded audio to a new file
    embedded_audio = wave.open(output_path, 'wb')
    embedded_audio.setparams(carrier.getparams())
    embedded_audio.writeframes(embedded_frames.astype(np.int16).tobytes())
    # Close files
    carrier.close()
    secret.close()
    embedded_audio.close()
    print(f"Embedding complete. Saved to {output_path}")

### Extraction


In [13]:
def extract_audio(embedded_path, secret_output_path, secret_length, num_bits=8):
    # Open the embedded audio file
    embedded = wave.open(embedded_path, 'rb')
    # Read audio data
    embedded_frames = np.frombuffer(embedded.readframes(
        embedded.getnframes()), dtype=np.int16)
    # Extract the secret audio
    mask = (1 << num_bits) - 1  # Mask to isolate the embedded bits
    extracted_bits = (embedded_frames & mask) << (16 - num_bits)
    secret_frames = extracted_bits[:secret_length*2]
    # Apply a low-pass filter to reduce noise
    secret_frames = np.convolve(secret_frames, np.ones(
        5)/5, mode='same')  # Simple smoothing (moving average)
    # Clip audio values to valid range
    secret_frames = clip_audio(secret_frames)
    # Write the secret audio to a new file
    secret_audio = wave.open(secret_output_path, 'wb')
    secret_audio.setparams(embedded.getparams())
    secret_audio.setnframes(secret_length)
    secret_audio.writeframes(secret_frames.astype(np.int16).tobytes())
    # Close files
    embedded.close()
    secret_audio.close()
    print(f"Extraction complete. Secret audio saved to {secret_output_path}")

## Implementation


### Input


In [14]:
# Carrier audio
carrier_audio = "cover.wav"
secret_audio = "secret.wav"  # Secret audio
embedded_audio = "embedded.wav"  # Output for the embedded audio

### Embed


In [15]:
embed_audio(carrier_audio, secret_audio, embedded_audio, num_bits=8)

Embedding complete. Saved to embedded.wav


# Encryption, Decryption, MAC Generation and Authentication

## Function Definitions

Encryption using the SHA3-256 digest of the final Ekert key.

In [16]:
def encryptAudio(key, inputFile, outputFile, outputAudio):
    with open(inputFile, 'rb') as inputFile:
        audioData = inputFile.read()
    header = b"Stego Audio"
    cipher = ChaCha20_Poly1305.new(key=key)
    cipher.update(header)
    nonce = cipher.nonce
    print("Nonce: ", nonce.hex())
    ciphertext, tag = cipher.encrypt_and_digest(audioData)
    print("Tag: ", tag.hex())
    with open(outputAudio, 'wb') as outputAudio:
        outputAudio.write(ciphertext)
    with open(outputFile, 'wb') as outputFile:
        outputFile.write(nonce+ciphertext+tag)

Applying random bit flip to encrypted audio in order to simulate middle-man attack

In [17]:
def addNoise(inputFile, noisyOutputFile):
    with open(inputFile, 'rb') as inputFile:
        noisyCiphertextBytes = bytearray(inputFile.read())
        for x in range(100000):
            indexToModify = np.random.randint(0, len(noisyCiphertextBytes)-1)
            noisyCiphertextBytes[indexToModify] ^= np.random.randint(
                0x00, 0xFF)
        with open(noisyOutputFile, 'wb') as noisyOutputFile:
            noisyOutputFile.write(noisyCiphertextBytes)

Decryption

In [18]:
def decryptAudio(key, inputFile, outputFile):
    with open(inputFile, 'rb') as inputFile:
        data = inputFile.read()
        nonce = data[:12]
        ciphertext = data[12:-16]
        tag = data[-16:]
    header = b"Stego Audio"
    cipher = ChaCha20_Poly1305.new(key=key, nonce=nonce)
    cipher.update(header)
    try:
        decryptedAudio = cipher.decrypt_and_verify(ciphertext, tag)
        with open(outputFile, 'wb') as outputFile:
            outputFile.write(decryptedAudio)
            print("Nonce: ", nonce.hex())
            print("Tag: ", tag.hex())
            print("Decryption and verification successful!")
    except ValueError:
        print("Verification failed!")

## Implementation

In [19]:
inputAudio = "embedded.wav"
outputAudio = "encrypted.wav"
output = "encrypted.bin"
decryptedAudio = "decrypted.wav"

In [20]:
encryptAudio(key, "embedded.wav", "encrypted.bin", "encrypted.wav")

Nonce:  0032c9e244e2b41e8297db0c
Tag:  c0d0aef87b3feb7c2b27f872de006f72


In [21]:
addNoise("encrypted.bin", "noisy.bin")
noisyOutput = "noisy.bin"

In [22]:
decryptAudio(key, output, decryptedAudio)

Nonce:  0032c9e244e2b41e8297db0c
Tag:  c0d0aef87b3feb7c2b27f872de006f72
Decryption and verification successful!


In [23]:
decryptAudio(key, noisyOutput, "outfile.bin")

Verification failed!


# Extraction

In [24]:
extracted_audio = "retrieved_audio1.wav"
# extract_audio('embedded.wav', 'output.wav', 8)

secret_audio_length = wave.open(
    secret_audio, 'rb').getnframes()  # Length of the secret audio
extract_audio(embedded_audio, extracted_audio, secret_audio_length)

Extraction complete. Secret audio saved to retrieved_audio1.wav


# Analysis